# 📖 Read Chromatographic Data

In [1]:
try:
    from chromatopy import ChromAnalyzer
except ImportError:
    print("Installing chromatopy...")
    pass

## Allotrope Simple Model (ASM) Format

The Allotrope Simple Model (ASM) format is the preferred format to be used with `chromatopy`. If not already done, you can batch process your chromatographic data with OpenChrom as [described](../../supported_formats/#spectrum-processing-with-openchrom-from-lablicate) to convert your data to the ASM format.

ASM data is read in with the `read_asm` function. This function reads chromatographic data from a directory containing Allotrope Simple Model (ASM) files. The measurements are assumed to be named alphabetically, allowing sorting by file name. The function requires the following parameters:

__Required Parameters__:
- `path`: The path to the directory containing the ASM files.
- `ph`: The pH value of the measurement.
- `temperature`: The temperature of the measurement.

__Optional Parameters__:
- `id`: A unique identifier for the `ChromAnalyzer` object. If not provided, the `path` is used as the ID.
- `name`: The name of the measurement. Defaults to "Chromatographic measurement".
- `reaction_times`: A list of reaction times, corresponding to each measurement in the directory.
- `time_unit`: The unit of the time values. If the reaction time and its corresponding unit are part of the file name of each individual ASM file, this argument can be omitted. Valid units include `sec`, `second`, `min`, `minute`, or `hour`.
- `temperature_unit`: The unit of the temperature. The default is Celsius (C).

__Notes__:
- The reaction time and the corresponding unit can be part of the file name of each individual ASM file. In this case, the `reaction_times` and `time_unit` arguments can be omitted. Examples of valid file names include: `m0 0 hours.json`, `fancyreaction_34sfgrr 12.5 minutes`, `m0 50mins.json`, `m0 50.12 seconds.json`


__Returns__:
The function returns a `ChromAnalyzer` object which is the main object used in `chromatopy` to work with.

In [2]:
from chromatopy import ChromAnalyzer

data_dir = "data/asm"

analyzer = ChromAnalyzer.read_asm(
    path=data_dir,
    ph=7.4,
    temperature=25,
)

## Other Supported Formats

#### Agilent ChemStation

Agilent ChemStation files can be read in with the `read_agilent` function. Please note that this function relies on the existence of either a `RESULTS.CSV` or a `RESULTS.TXT` file in the `*.D` directory, which can be generated by the Agilent ChemStation software. Thus, it is recommended to use OpenChrom to convert your data to the ASM format.

In [5]:
from chromatopy import ChromAnalyzer, minute

data_dir = "data/agilent_txt"

analyzer = ChromAnalyzer.read_agilent(
    path=data_dir,
    ph=7.4,
    temperature=25,
)

ValidationError: 1 validation error for AgilentTXTReader
  Value error, No files in found in the directory data/agilent_txt. [type=value_error, input_value={'dirpath': 'data/agilent...Chemistry/chromatopy'})}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

### Shimadzu

Shimadzu `*.txt` files can be read in with the `read_shimadzu` function. This function reads chromatographic data as the previously described functions.  
In this dataset, the corresponding reaction times for each measurement are manually specified and the respective `time_unit` is imported.

In [9]:
from chromatopy import ChromAnalyzer

data_dir = "data/shimadzu"

analyzer = ChromAnalyzer.read_shimadzu(
    path=data_dir,
    ph=7.4,
    temperature=25,
    reaction_times=[0, 5, 10, 15, 20, 25, 30, 35, 40],
    time_unit=minute,
)

Output-sample 1
Output-sample 0
Output-sample 2
Output-sample 3
Output-sample 7
Output-sample 6
Output-sample 4
Output-sample 5
Output-sample 8
